# Introduction to the [Tensorflow](https://www.tensorflow.org/tutorials) framework with the Keras API. 

We will work with the MNIST dataset of hand-written digits. The main task is to recognise the digit wisible on image. We will perfomr following actions: 
* image classification using logistic regression 
* image classification using multi-layer dense neural networks
* examine various network depths, activation functions and optimizers

We will be using a high level [Keras](https://keras.io/getting_started/intro_to_keras_for_researchers/) Application Programming Interface (API)

# Environment preparation

* importing of necessary libraries
* if given library is not present in software environment it can be installed with "magic" commands:

```
! pip install matplotlib
```

In [ ]:
#An eycandy colored text can be printer using package colored
from termcolor import colored

#The Tensorflow library 
import tensorflow as tf

#Package installation command
!pip install matplotlib

#The plotting library
from matplotlib import pyplot as plt

import numpy as np

## Data loading

* load the test data from a library of benchmark dataset
* print basic information on imported data

In [ ]:
(DATA0, TARGET0), (DATA1, TARGET1) = tf.keras.datasets.mnist.load_data()

print(colored("The training dataset:","blue"))
print("Features. type: {}, shape: {}, min/max: {}/{}".format(type(DATA0), DATA0.shape, DATA0.min(), DATA0.max()))
print("Labels. type: {}, shape: {}, min/max: {}/{}".format(type(TARGET0), TARGET0.shape, TARGET0.min(), TARGET0.max()))

print("\n")
print(colored("The validation dataset:","blue"))
print("Features. type: {}, shape: {}, min/max: {}/{}".format(type(DATA1), DATA1.shape, DATA1.min(), DATA1.max()))
print("Labels. type: {}, shape: {}, min/max: {}/{}".format(type(TARGET1), TARGET1.shape, TARGET1.min(), TARGET1.max()))

These are ordinary numpy arrays. There are 60 000 samples in the training dataset and 10 000 in the validation dataset. <br> The targets are integer numbers from 0 to 9 labeling the digits. The data are 28x28 grayscale images with values of type uint8, ranging from 0 to 255.

## Visual data inspection

* display the first image from the training dataset
* display 100th image from 
* find index of first image with "9" in the trainin dataset and display relevant image

In [ ]:
plt.subplot(131) #nrows ncolumns index
plt.imshow(DATA0[0]);

plt.subplot(132)
plt.imshow(DATA0[99]);

index  = np.argmax(TARGET0==9)
print("Index of first \"9\" in the training dataset is {}".format(index))

plt.subplot(133)
plt.imshow(DATA0[index]);

## <span style='color:red'> Please: </span> 

* plot image of the image of the first "7" in the validation dataset

In [ ]:
index  = ...
print("Index of first \"7\" in the training dataset is {}".format(index))
...

## Data preprocessing

Ofter tha input data has to be preprocessed to be useful in the training process. Dense networks take flat vectors of floating-point features and are best suited for inputs of the order of 1 rather than 255. 

* flatten images - change the 2D arrays into a 1D vector. Remember we have many such images, so the shape change is:

```
(nExamples, nX, nY) -> (nExamples, nX*nY)
```

* rescale the input values fo [0,1] range

In [ ]:
(DATA0, TARGET0), (DATA1, TARGET1) = tf.keras.datasets.mnist.load_data()

print(colored("Before preprocessing.","blue"), "Pixel (14,14) from the first example of training data: {}".format(DATA0[0,14,14]))

DATA0 = DATA0.reshape(-1, 28 * 28)
DATA0 = DATA0.astype('float32')
DATA0 = DATA0/float(DATA0.max())

print(colored("After preprocessing.","blue"), "Pixel (14,14) from the first example of raining data: {}".format(DATA0[0,14*28+14]))

#Combine all sten into a single line
DATA1 = DATA1.reshape(-1, 28 * 28).astype('float32') / 255.

## A logistic regression model

Suppose we have $I$ data samples of $J$ features each and want to classify them to $K$ classes. Let $X_{ij}$ denote feature $j$ of sample $i$. In logistic regression, we first calculate the logits: 

$$A_{ik}=b_k+\sum_jX_{ij}w_{jk}$$ 

where $b_k$ and $w_{jk}$ are called biases and weights. Then we calculate the probabilities that sample $i$ belongs to class $k$ according to the softmax formula: 

$$P_{ik}=\frac{\exp(A_{ik})}{\sum_{k'}\exp(A_{ik'})}$$ 

We then classify each sample to the class with the highest probability. The biases and weights are determined by training the classifier on a dataset $X_{ij}$ where sample $i$ belongs to class $k_i$. This is done by minimizing the cross-entropy loss 

$$L=-\sum_{ik}\delta_{kk_i}\log P_{ik}$$

by using a variant of stochastic gradient descent. 

Let us implement the above model using the Tensorflow Keras interface.

* the model is defines as a set of subsequent layers
* dense layer is a set of "neurons" that each take a vectorial input, and return a single value. With default setting this corresponds to $A_{i,j}$ with fixed j.
  The $k$ index runs over examples provided at the input
* we need a single neuron for each digi, to the dense layesr has size 10  
* the output of all neurons has to be normalised using the $P_{ik}$ formula - this is done by the ```tf.keras.layers.Softmax``` layer  

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()])

* the activation functions, like ```softmax```, ```sigmoid```, ```relu```, etc., can be treated as separate layers or equivalently added at the outputs of the dense layers:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation = 'softmax')])

* the weighs of the model are now at some random values, sicne the model has not been trained yet, 
* we can pass the validation dataset, but the results will be random as well
* the result of the model is a 2D array(tensor): `(nExamples, nNeurons)`
* **Note** the output type is `tf.Tensor`. Sometimes is is necessary to cast it to numpy array (we will not use this today):

```
PROBABILITY1 = PROBABILITY1.numpy()
```

In [ ]:
PROBABILITY1 = model(DATA1)

print(colored("Model output shape:","blue"),PROBABILITY1.shape)
print(colored("Digits probabilities for the first validation example:","blue"), PROBABILITY1[0])
print(colored("Maximum probability is for digit:","blue"), np.argmax(PROBABILITY1[0]))
print(colored("The first example label is:","blue"),TARGET1[0])

## <span style='color:red'> Please: </span> 

Check if the probality normalisation is correct:

* use `tf.math.reduce_sum(input_tensor, axis)` function to sum all columns of the output probability for all the xemaples
* print the result for the first example

In [ ]:
SUM1 = ...
print(colored("Sum of the probabilities for the first example:","blue"),SUM1[0])

## <span style='color:red'> Please: </span> 

* create the model again print the probabilities for the first exmample in the cell below
* execute the cell a few times
* observe is the model output changes

In [ ]:
model = ...

PROBABILITY1 = ...

print(colored("Model output shape:","blue"), ...)
print(colored("Digits probabilities for the first validation example:","blue"), ...)
print(colored("Maximum probability is for digit:","blue"), ...)
print(colored("The first example label is:","blue"), ...)

## Performace metric - accuracy

We have been looking at the output of the model for single exmaple. We need to have a metric stating the model performance on a whole set of examples.
Here we will use ```accuracy``` - fraction of correctly assigned labels

* create w vector of model labels - indices where the model prediction is maximal
* create a vector of ```True/False``` for correct/incorrect label assignement
* calculate the mean value of this vector - this is the accuracy in our case
* **what is the expected accuracy for random guessing of the digit?**

In [ ]:
LABEL1 = np.argmax(PROBABILITY1, axis=1)
ACCURACY1 = (LABEL1 == TARGET1)

print(colored("Is the model answer correct?","blue"),ACCURACY1)
print(colored("Fraction of correct model answers:","blue"),ACCURACY1.mean())

## Model training

To find the model weighs that will be the "best" we need:

* **optimizer** - an algoritm that will adapt weights to minimize the . We will use `stochastic gradient descent (sgd)`
* **loss function** - the definition of "the best" - the function that will be minimalize by optimizer We will use `sparse categorical crossentropy`
* **metris** - a metric of interest for the model user - we will use `accuracy`

In [ ]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics='accuracy')

Once the model is associated with a loss function and a metric, calculate their values on any dataset by calling a built-in method:

* the evaluation runs in batches. Default batch size is 32
* a single run througn the whole data set is an `epoch`
* for the training data we have 60000/32 = 1875 steps per epoch
* loss anf metrics are averaged over the data processed, so it changes during looping through the dataset

In [ ]:
model.evaluate(DATA0, TARGET0)
model.evaluate(DATA1, TARGET1)

The accuracy on the validation dataset is indeed equal to what we calculated manually. Now train the model on the training dataset. Do only 10 epochs for now:

In [ ]:
model.fit(x=DATA0, y=TARGET0, epochs = 10)

The accuracy on the training dataset may not be representative for new samples due to the so-called overfitting.

* to get a more reliable estimation of the model accuracy, evaluate metrics on a validation dataset
* change the batch size and do more epochs to perform a full training. Larger batch size will allow to better explot the GPU capabilities, as more data will be processed in parallel
* use ``%%time`` magic command to measurte the cell exetuction time
* use `verbose=0` to silent the printout. It is much more convenient to plot the metric values
* keep the output of the `model.fit()` method in a variable. The ouput is training history, and can by used for plotting the metric values

In [ ]:
%%time
history = model.fit(DATA0, TARGET0, batch_size = 128, epochs = 10, validation_data = (DATA1, TARGET1), verbose=0);

Note that with the batch of 128, a single epoch executes roughly twice faster than with the default batch of 32. Why? What may be the other consequences of changing the batch size from 32 to 128?

Even though the batch of 128 is 3 times larger than the default batch of 32, Tensorflow is optimized so that processing the larger batch takes only 1.5 more time than processing the smaller one (3ms vs 2ms). But there are 3 times less large batches per epoch than there are small ones, which translates to 3 times less minimization steps. This gives 3 / 1.5 = 2 times less time per epoch with the larger batch (2s vs 4s). Less minimization steps per epoch imply that more epochs may be needed to achieve the same result. On the other hand, larger batch gives a better estimation of the real gradient so the minimization is less noisy.

Although this is not absolutely necessary, evaluate the model on the training and validation dataset:

In [ ]:
model.evaluate(DATA0, TARGET0);
model.evaluate(DATA1, TARGET1);

**Question.** The validation accuracy displayed after the last training epoch is equal to the validation accuracy evaluated after completing the training. This is not the case for the accuracy on the training dataset. Why?

**Solution.** Accuracy is calculated as a mean over all batches in the concerned dataset. The validation accuracy displayed during training is calculated after completing each epoch. But the training accuracy displayed during training is not calculated in such a way, because this would require iterating over the entire training dataset after completing each epoch and could be time-consuming. Instead, the training accuracy displayed during training is calculated during the epoch. But during the epoch, a minimization step is performed after each batch, so subsequent batches correspond to different model weights. Consequently, the displayed training accuracy does not correspond to any fixed set of model weights and is only an approximation to the real accuracy on the training dataset.

In the following, by the validation accuracy of a model, we will always mean the maximum value listed during training, even if it is not in the last epoch. This is because Tensorflow can stop training after any epoch and thus produce a model with validation accuracy corresponding to that epoch. 

* print a summary of the model architecture
* create a plot with layers description

In [ ]:
model.summary()
tf.keras.utils.plot_model(model, 'ML_model.png', show_shapes=True)

**Question** Why does the number of weights equal 7 850?

**Solution.** To produce its output, a dense layer multiplies the input by a matrix whose one dimension is the number of inputs and the other is the number of outputs. That matrix has 784 * 10 = 7 840 elements in this case. Then, the layer adds a bias to each output, so there are 10 biases. Together, there are 7 840 + 10 = 7 850 weights.

**Task.** In this example, the training and validation accuracies are very close to each other, which means that there is almost no overfitting. Why?

**Solution.** Because there are much more samples in the training dataset than there are model weights. So, the problem is overdetermined and there are not enough degrees of freedom to adjust the model too much to the training dataset.

## Training history

It is good to monitor the evolution of the loss function and the metric duriong the training

* inspect the content of the `history` variable initializez by the output of the `model.fit()` method
* plot the accuracy as a function of the epoch numer 

In [ ]:
print(colored("Content of the history object:", "blue"),history.history.keys())
print(colored("Accuracy on the traiing dataset:","blue"), history.history['accuracy'])

def plotTrainingHistory(history):

    fig, axes= plt.subplots(1,2,figsize=(10,5))
    history = history.history
    axes[0].plot(history['accuracy'])
    axes[0].plot(history['val_accuracy'])
    axes[0].set_ylabel('accuracy')
    axes[0].set_xlabel('epoch')
    axes[0].legend(['train', 'validation'], loc='upper left')

    axes[1].set_ylabel('loss function')
    axes[1].set_xlabel('epoch')
    axes[1].legend(['train', 'validation'], loc='upper left')    
    
plotTrainingHistory(history)   

## <span style='color:red'> Please: </span> 

* finish the `plotTrainingHistory(history)` function by adding a plot for the loss function evolution  

In [ ]:
def plotTrainingHistory(history):

    ...
    ...
    ...
    
plotTrainingHistory(history)      

## Model extension

* add a second layer, **a hidden layer**, between the input, and output layers
* choosing the hidden layers parameters is part of the ML art. A rule of thumb for the number of neurons in that layer may be, the geometric mean of the number of inputs and outputs, that is:

$$
\sqrt{10 \cdot 784} = 89
$$

or the next power of two, that is, 128. 
* use the sigmoid activation function for the hidden layer

In [ ]:
%%time 

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation = 'sigmoid'),
    tf.keras.layers.Dense(10, activation = 'softmax')])

model.compile('sgd',
              'sparse_categorical_crossentropy',
              'accuracy')

history = model.fit(DATA0, TARGET0, batch_size = 128, epochs = 100, validation_data = (DATA1, TARGET1), verbose=0)

model.summary()
model.evaluate(DATA0, TARGET0);
model.evaluate(DATA1, TARGET1);
tf.keras.utils.plot_model(model, 'ML_model.png', show_shapes=True)
plotTrainingHistory(history)   

**Task.** Explain the number of weights in this network.

**Solution.** The first layer has 784 inputs and 128 outputs, so 784 * 128 + 128 = 100 480 weights. The second layer has 128 inputs and 10 outputs, so 128 * 10 + 10 = 1 290 weights. Together, there are 100 480 + 1 290 = 101 770 weights.

Adding the second layer improved the accuracy surprisingly little. 

* change the activation function of the added layer to `relu`, which is one of the most used:

In [ ]:
%%time

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'softmax')])

model.compile('sgd',
              'sparse_categorical_crossentropy',
              'accuracy')

history_sgd = model.fit(DATA0, TARGET0, batch_size = 128, epochs = 100, validation_data = (DATA1, TARGET1), verbose=0)
model.summary()
model.evaluate(DATA0, TARGET0);
model.evaluate(DATA1, TARGET1);
plotTrainingHistory(history_sgd)   

Now the improvement is very significant. The difference is because sigmoid saturates on both sides, yields vanishing gradients there, and effectively stucks the minimization. Relu does not saturate on the positive side and causes no such problems.

**Question.** The training takes less physical time with the relu activation. Why?

**Solution.** Relu does not contain nonlinear functions, like exponents, and is therefore faster to calculate and differentiate.

* change the optimizer algorithm to `Adaptive Momentum (Adam)`, which is one of the best and most used:

In [ ]:
%%time

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'softmax')])

model.compile('adam',
              'sparse_categorical_crossentropy',
              'accuracy')

history_adam = model.fit(DATA0, TARGET0, batch_size = 128, epochs = 100, validation_data = (DATA1, TARGET1), verbose=0)
model.summary()
model.evaluate(DATA0, TARGET0);
model.evaluate(DATA1, TARGET1);

print(colored("SGD optimizer final validation accuracy:","blue"), history_sgd.history["val_accuracy"][-1])
plotTrainingHistory(history_sgd)

print(colored("ADAM optimizer final validation accuracy:","blue"), history_adam.history["val_accuracy"][-1])
plotTrainingHistory(history_adam)

The final validation accuracy is only slightly higher but Adam achieved it in a much smaller number of epochs than SGD. From now on, we will always use the Adam optimizer and the relu activation function, and 32 epochs.

It is impressive that such a simple two-layer network correctly classifies 100% of the training dataset. 
**This means that the lower accuracy on the validation dataset is uniquely due to overfitting.**

**Question.** Why is there a significant overfitting here?

**Solution.** Because the number of model weights is significantly higher than the number of training samples. The problem is now underdetermined and gives room for excessive adjustment of the model weights to this particular training dataset.

Since there is overfitting caused by the already large number of model weights, it is dubious whether adding a third layer would still improve things, because it will increase the number of weights even more. Check this by adding a third layer. 

## <span style='color:red'> Please: </span> 

* create a model with three layers with 256, 65, 10 neutrons
* the last layer is the output layer, so it should have `activation = 'softmax'
* other layers should have activation = 'relu'
* use `adam` optimizer
* use `batch_size = 32`, and run the optimization for `epochs = 32`
* submit the accuaracy wih three digits precision: 0.XXX on the validation dataset [here](https://docs.google.com/forms/d/1ZqW4bwP9jVepDUWlHnBkWzrlzQrVkyujjnsrNxoBPrs)

In [ ]:
model = ...

model.compile(...)

history = model.fit(...)
model.summary()
model.evaluate(DATA0, TARGET0);
model.evaluate(DATA1, TARGET1);
plotTrainingHistory(history)   

Adding the third layer improved the validation accuracy only slightly. But this is still an achievement because the closer we are to 100%, the more difficult it is to gain accuracy. Note that even a slight increase in accuracy leads to a significant reduction in the error rate, which is complementary to accuracy. Here for instance, increasing the accuracy from 0.980 to 0.985 reduces the error rate from 2% to 1.5%, that is, by a factor of 4/3.

## <span style='color:red'> Please: </span> 

* check that adding a fourth dense layer does not further increase the validation accuracy.

In [ ]:
%%time 

model = ...

model.compile(...)

history = model.fit(...)

model.summary()
model.evaluate(DATA0, TARGET0);
model.evaluate(DATA1, TARGET1);
plotTrainingHistory(history)   

Since adding more dense layers does not further increase the validation accuracy, we must resort to other means. 

There are two paths:

* model regularisation - dedicated techniques to reduce overfitting, 
* model modification - use of different type of layers. The `convolutional layers` are specifically designed for image analyses.

..but this is a story for an medium level ML Workshop...

## Model deployment

After obtaining a satifcatory model, one still need to deliver it to "client". Often this requires some additional work.

* please test the model on your own digits.

In [ ]:
import os 

def testModelOnMyDigits(model):    
    #Code created by M.Fila@UW
    if not os.path.isdir("colab_freehands"):
        !git clone https://github.com/m-fila/colab_freehands.git

    from colab_freehands.canvas import Canvas  
    canvas = Canvas(line_width=2)
    example = (
        canvas.to_array(size=(20, 20), margin=(4, 4), dtype=np.float32, weighted=True) / 255
    )
    predictions = model(np.expand_dims(example, (0, -1)))
    plt.imshow(example, cmap="gray")
    plt.show()
    print(
        "Predicted class: {} ({:.0f}%)".format(
            np.argmax(predictions), np.max(predictions) * 100
        )
    )
    
testModelOnMyDigits(model)    